In [186]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Below is the function to change all the column headers to snake_case.

Note: I have only considered space,/ and - after looking one or two files.




In [187]:
from pyspark.sql.functions import col
def rename_cols(rename_df):
    for column in rename_df.columns:
        new_column = column.replace(' ','_').replace('/','_').replace('-','_')
        rename_df = rename_df.withColumnRenamed(column, new_column)
    return rename_df

Below is the main code to read the csv files from datastore that are related to the theme "Hospitals" and load them to storage path.

Note: As I dont have any work environment of Azure to load the files, I have saved all the files from data store to my local path.


In [188]:
import gettext
import json
import requests
import pandas as pd
import os

api_url = f'https://data.cms.gov/provider-data/api/1/metastore/schemas/dataset/items'

getjsondata = requests.get(api_url)

response_json = json.loads(getjsondata.text)

#Looping through all the json records
for todoFile in response_json:

  #Looking only for the theme Hospitals
  if str(todoFile["theme"]) == "['Hospitals']":

    #Extracting only the filename from the URL
    fileName = os.path.basename(todoFile["distribution"][0]["downloadURL"])


    #Extracting ModifiedDate column to append to filename and also to identity if the file has already loaded or not
    modifiedDate = todoFile["modified"]

    #updating the modifiedDate format to YYYYMMDD
    modifiedDateFormatted = modifiedDate.replace("-", "")

    #Generating the fileName for saving YYYYMMDD_FileName.csv
    fileNameFinal =  modifiedDateFormatted + "_" + fileName

    #Reading the csv file using pandas dataframe
    initialDf = pd.read_csv(todoFile["distribution"][0]["downloadURL"])

    #Checking if the file already exists or loaded. We will only load the file if it is not there in our folder
    if not os.path.exists("/Users/vinay/CMSData/" + fileNameFinal):
      #Note : have converted panda dataframe to spark dataframe for renaming columns. I did this quick turnaround as I have already spent most of the time on this.
      spark_df = spark.createDataFrame(initialDf)

      #Calling the function to change all the column headers to snake_case.
      spark_df1 = rename_cols(spark_df)

      #Finally writihng the file to the storage location.
      spark_df1.write.option("header",True).mode("overwrite").csv("/Users/vinay/CMSData/" + fileNameFinal)
    else:
      print("File Already exists")



<ipython-input-188-123db81bc451>:33: DtypeWarning: Columns (12,14,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  initialDf = pd.read_csv(todoFile["distribution"][0]["downloadURL"])
<ipython-input-188-123db81bc451>:33: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  initialDf = pd.read_csv(todoFile["distribution"][0]["downloadURL"])


In [189]:
#Took one randome file to read/verify the contents after loading
spark.read.csv("/Users/vinay/CMSData/20241010_Timely_and_Effective_Care-State.csv",header = True).show()

+-----+--------------------+--------------------+--------------------+-------------+--------+----------+----------+
|State|           Condition|          Measure_ID|        Measure_Name|        Score|Footnote|Start_Date|  End_Date|
+-----+--------------------+--------------------+--------------------+-------------+--------+----------+----------+
|   AK|Healthcare Person...|        HCP_COVID_19|Percentage of hea...|          9.2|     NaN|10/01/2023|12/31/2023|
|   AK|Healthcare Person...|               IMM_3|Healthcare worker...|           80|     NaN|10/01/2023|03/31/2024|
|   AK|Emergency Department|              OP_18b|Average (median) ...|          140|  25, 26|01/01/2023|12/31/2023|
|   AK|Emergency Department|     OP_18b_HIGH_MIN|Average time pati...|          150|  25, 26|01/01/2023|12/31/2023|
|   AK|Emergency Department|      OP_18b_LOW_MIN|Average time pati...|          134|  25, 26|01/01/2023|12/31/2023|
|   AK|Emergency Department|   OP_18b_MEDIUM_MIN|Average time pati...|  

In [190]:
#Took one more randome file to read/verify the contents after loading
spark.read.csv("/Users/vinay/CMSData/20241018_HCAHPS-Hospital.csv",header = True).show()

+-----------+--------------------+-------------+---------+-----+--------+-------------+----------------+--------------------+--------------------+-------------------------+--------------------------+-----------------------------------+---------------------+------------------------------+------------------------+---------------------------+------------------------------------+----------------------------+-------------------------------------+----------+----------+
|Facility_ID|       Facility_Name|      Address|City_Town|State|ZIP_Code|County_Parish|Telephone_Number|   HCAHPS_Measure_ID|     HCAHPS_Question|HCAHPS_Answer_Description|Patient_Survey_Star_Rating|Patient_Survey_Star_Rating_Footnote|HCAHPS_Answer_Percent|HCAHPS_Answer_Percent_Footnote|HCAHPS_Linear_Mean_Value|Number_of_Completed_Surveys|Number_of_Completed_Surveys_Footnote|Survey_Response_Rate_Percent|Survey_Response_Rate_Percent_Footnote|Start_Date|  End_Date|
+-----------+--------------------+-------------+---------+-----+

In [191]:
#Took one more randome file to read/verify the contents after loading
spark.read.csv("/Users/vinay/PythAssessment/20241010_Complications_and_Deaths-Hospital.csv",header = True).show()

+-----------+--------------------+--------------------+-----------+-----+--------+-------------+----------------+-------------+--------------------+--------------------+--------------+-------------+--------------+---------------+--------+----------+----------+
|Facility_ID|       Facility_Name|             Address|  City_Town|State|ZIP_Code|County_Parish|Telephone_Number|   Measure_ID|        Measure_Name|Compared_to_National|   Denominator|        Score|Lower_Estimate|Higher_Estimate|Footnote|Start_Date|  End_Date|
+-----------+--------------------+--------------------+-----------+-----+--------+-------------+----------------+-------------+--------------------+--------------------+--------------+-------------+--------------+---------------+--------+----------+----------+
|     251319|HOLMES COUNTY HOS...|239 BOWLING GREEN...|  LEXINGTON|   MS|   39095|       HOLMES|  (662) 834-1321|       PSI_03| Pressure ulcer rate|       Not Available| Not Available|Not Available| Not Available|  No